# Linear Regression

Linear regression fits a straight line to data. The model assumes a linear relationship
y=w1x1+w2x2+⋯+b y=w1​x1​+w2​x2​+⋯+b
between features and target. For one feature, this is $y = mX + b$
kdnuggets.com
. The parameters (weights $w$ and bias $b$) are chosen to minimize the average squared error (MSE) between predictions and actual values:
MSE(w,b)=1N∑i=1N(yi−y^i)2,y^i=wTxi+b.MSE(w,b)=N1​∑i=1N​(yi​−y^​i​)2,y^​i​=wTxi​+b.
Minimizing MSE can be done by solving the normal equations (closed-form) or by gradient descent. For example, given data of hours trained vs points scored, we can iteratively update weights by computing gradients
kdnuggets.com
:

In [ ]:
import os
import sys
import subprocess
import textwrap

def _run(cmd):
    try:
        out = subprocess.check_output(cmd, stderr=subprocess.STDOUT, text=True)
        return True, out.strip()
    except Exception as e:
        return False, f"{type(e).__name__}: {e}"

def env_snapshot():
    keys = [
        "JAX_PLATFORM_NAME", "JAX_PLATFORMS", "CUDA_VISIBLE_DEVICES",
        "XLA_FLAGS", "NVIDIA_VISIBLE_DEVICES", "NVIDIA_DRIVER_CAPABILITIES"
    ]
    print("\n=== ENV SNAPSHOT (selected) ===")
    for k in keys:
        v = os.environ.get(k)
        print(f"{k}={v!r}")
    print("===============================\n")

def check_nvidia_smi():
    ok, out = _run(["nvidia-smi"])
    print("=== NVIDIA-SMI ===")
    if ok:
        print(out.splitlines()[0])
        print("GPU is visible to the container/process.")
    else:
        print("No GPU visible (nvidia-smi failed).")
        print(out)
    print("==================\n")
    return ok

def check_pytorch():
    print("=== PyTorch ===")
    try:
        import torch
        print(f"torch.__version__={torch.__version__}")
        print(f"torch.version.cuda={torch.version.cuda}")
        print(f"torch.cuda.is_available()={torch.cuda.is_available()}")
        if torch.cuda.is_available():
            print(f"torch.cuda.device_count()={torch.cuda.device_count()}")
            print(f"current device name: {torch.cuda.get_device_name(0)}")
            a = torch.randn(1024, 1024, device="cuda")
            b = torch.randn(1024, 1024, device="cuda")
            c = (a @ b).sum().item()
            print("small matmul check (CUDA): OK")
        else:
            print("CUDA not available in PyTorch.")
    except Exception as e:
        print("PyTorch check failed:", repr(e))
    print("==============\n")

def check_jax():
    print("=== JAX ===")
    try:
        import jax, jaxlib  # noqa
        print(f"jax.__version__={jax.__version__}")
        try:
            import jaxlib
            print(f"jaxlib.__version__={jaxlib.__version__}")
        except Exception as e:
            print("Could not import jaxlib:", repr(e))

        # Try to list devices safely
        try:
            devs = jax.devices()
            if not devs:
                print("No JAX devices found.")
            else:
                for d in devs:
                    print(f"Device: kind={d.device_kind}, platform={d.platform}, id={d.id}")
            # Functional smoke test on default backend
            from jax import numpy as jnp
            x = jnp.ones((1024,1024), dtype=jnp.float32)
            y = (x @ x.T).sum()
            _ = y.block_until_ready()
            print("small matmul check (JAX default backend): OK")
        except RuntimeError as re:
            # Decode the common “Unknown backend: 'gpu'” case
            msg = str(re)
            print("jax.devices() raised RuntimeError:", msg)
            if "Unknown backend: 'gpu'" in msg:
                print(textwrap.dedent("""
                    HINT: JAX was forced to use the 'gpu' platform but no GPU backend is present.
                    Check for:
                    • JAX_PLATFORM_NAME or JAX_PLATFORMS env vars forcing 'gpu'
                    • CPU-only jaxlib installed instead of CUDA-enabled wheels
                    • Container not launched with GPU (e.g., --gpus all)
                    """).strip())
        except Exception as e:
            print("jax.devices() failed:", repr(e))
    except Exception as e:
        print("JAX import failed:", repr(e))
    print("===========\n")

def check_tensorflow():
    print("=== TensorFlow ===")
    try:
        import tensorflow as tf
        print(f"tf.__version__={tf.__version__}")
        try:
            gpus = tf.config.list_physical_devices("GPU")
            print(f"tf GPUs: {gpus}")
            if gpus:
                # Minimal compute test (graphless)
                with tf.device("/GPU:0"):
                    a = tf.random.normal((1024,1024))
                    b = tf.random.normal((1024,1024))
                    c = tf.reduce_sum(tf.matmul(a, b))
                _ = c.numpy()
                print("small matmul check (TensorFlow GPU): OK")
            else:
                print("No TensorFlow GPU devices found.")
        except Exception as e:
            print("TensorFlow device query failed:", repr(e))
    except Exception as e:
        print("TensorFlow import failed:", repr(e))
    print("==================\n")

def main():
    env_snapshot()
    gpu_visible = check_nvidia_smi()
    check_pytorch()
    check_jax()
    check_tensorflow()

    if not gpu_visible:
        print(textwrap.dedent("""
        SUMMARY: GPU NOT VISIBLE IN CONTAINER/PROCESS
        - Ensure Docker is started with GPU access:
            docker run --gpus all ...
          or in docker-compose.yml:
            services:
              app:
                gpus: all
                environment:
                  - NVIDIA_VISIBLE_DEVICES=all
                  - NVIDIA_DRIVER_CAPABILITIES=compute,utility
        - On Windows, ensure WSL2 + NVIDIA drivers + NVIDIA Container Toolkit.
        """).strip())

if __name__ == "__main__":
    main()



=== ENV SNAPSHOT (selected) ===
JAX_PLATFORM_NAME=''
JAX_PLATFORMS=None
CUDA_VISIBLE_DEVICES='0'
XLA_FLAGS='--xla_force_host_platform_device_count=1'
NVIDIA_VISIBLE_DEVICES='all'
NVIDIA_DRIVER_CAPABILITIES='compute,utility'

=== NVIDIA-SMI ===
Thu Aug 28 20:31:54 2025       
GPU is visible to the container/process.

=== PyTorch ===


In [ ]:
#!/usr/bin/env python3
# linear_regression_gradient_descent.py

"""
Linear Regression with Gradient Descent
---------------------------------------
This script demonstrates a simple linear regression model trained
using gradient descent on example data (hours practiced vs points scored).

Key Concepts:
  • Initialization: Starting weight w and bias b; in convex problems
    (like linear regression with MSE) the final solution is independent
    of the start, but the convergence speed can vary.
  • Bias term b: Model intercept, i.e. prediction when x=0.
  • Shapes: 
      X is (N×1), w is (1,), y_pred is (N,), error is (N,),
      dw is (1,), db is scalar.
  • Gradient descent update:
      dw = (1/N) * Xᵀ·(y_pred - y)
      db = (1/N) * Σ(y_pred - y)
      w ← w - lr·dw
      b ← b - lr·db

Adjustable via CLI flags:
  --lr        Learning rate (default 0.1)
  --iters     Number of iterations (default 1000)
  --seed      Random seed for initializing w (default 42)
"""

import numpy as np
import argparse
import sys

def parse_args():
    parser = argparse.ArgumentParser(
        description="Linear regression via gradient descent"
    )
    parser.add_argument("--lr",    type=float, default=0.1,  help="Learning rate")
    parser.add_argument("--iters", type=int,   default=1000, help="Training iterations")
    parser.add_argument("--seed",  type=int,   default=42,   help="Random seed")

    # Ignore unknown args (e.g., Jupyter’s -f flag) to avoid parse errors
    args, unknown = parser.parse_known_args()  
    return args

def main(lr, n_iters, seed):
    # ---------------------------------------------------------------------
    # 1. Create the data
    # ---------------------------------------------------------------------
    # Feature matrix X: hours practiced (5 examples) as a column vector (shape: 5×1)
    X = np.array([[1], [2], [3], [4], [5]])
    # Target vector y: points scored (shape: (5,))
    y = np.array([1.5, 3.0, 4.0, 4.5, 5.0])
    N = len(y)  # number of samples

    # ---------------------------------------------------------------------
    # 2. Initialize parameters
    # ---------------------------------------------------------------------
    # Set RNG seed for reproducibility of w initialization
    rng = np.random.default_rng(seed)
    # Weight vector w (shape: (1,)), random start
    w = rng.random(size=1)
    # Bias scalar b, start at zero (intercept term)
    b = 0.0

    print(f"Initial weight: {w[0]:.4f}, bias: {b:.4f} (seed={seed})")
    print(f"Training for {n_iters} iterations with lr={lr}\n")

    # ---------------------------------------------------------------------
    # 3–6. Training loop: forward pass, compute gradients, update params
    # ---------------------------------------------------------------------
    for i in range(1, n_iters + 1):
        # 3. Forward pass: predictions (shape: (5,))
        y_pred = X.dot(w) + b

        # 4. Compute gradients of MSE loss:
        #    dw = (1/N) * Xᵀ·(y_pred - y)   → shape: (1,)
        #    db = (1/N) * Σ(y_pred - y)     → scalar
        error = y_pred - y
        dw = (1 / N) * X.T.dot(error)
        db = (1 / N) * np.sum(error)

        # 5. Parameter update: move against the gradient
        w -= lr * dw
        b -= lr * db

        # Print progress periodically (every 10% of iterations)
        if i % max(1, n_iters // 10) == 0 or i == 1:
            loss = (error**2).mean()
            print(f"Iter {i:4d}/{n_iters}, loss={loss:.4f}, w={w[0]:.4f}, b={b:.4f}")

    # ---------------------------------------------------------------------
    # 7. Inspect the trained model
    # ---------------------------------------------------------------------
    print("\nTraining complete.")
    print(f"Final weight: {w[0]:.4f}, bias: {b:.4f}\n")

    # Compare predictions vs. actual values
    print("Predictions vs. Actual:")
    for x_val, actual in zip(X.flatten(), y):
        pred = w * x_val + b
        print(f"  x = {x_val:>1}, predicted = {pred[0]:.2f}, actual = {actual:.2f}")

if __name__ == "__main__":
    args = parse_args()
    main(lr=args.lr, n_iters=args.iters, seed=args.seed)


Initial weight: 0.7740, bias: 0.0000 (seed=42)
Training for 1000 iterations with lr=0.1

Iter    1/1000, loss=1.7402, w=1.1726, b=0.1278
Iter  100/1000, loss=0.1007, w=0.8981, b=0.8763
Iter  200/1000, loss=0.0952, w=0.8587, b=1.0184
Iter  300/1000, loss=0.0950, w=0.8516, b=1.0443
Iter  400/1000, loss=0.0950, w=0.8503, b=1.0490
Iter  500/1000, loss=0.0950, w=0.8501, b=1.0498
Iter  600/1000, loss=0.0950, w=0.8500, b=1.0500
Iter  700/1000, loss=0.0950, w=0.8500, b=1.0500
Iter  800/1000, loss=0.0950, w=0.8500, b=1.0500
Iter  900/1000, loss=0.0950, w=0.8500, b=1.0500
Iter 1000/1000, loss=0.0950, w=0.8500, b=1.0500

Training complete.
Final weight: 0.8500, bias: 1.0500

Predictions vs. Actual:
  x = 1, predicted = 1.90, actual = 1.50
  x = 2, predicted = 2.75, actual = 3.00
  x = 3, predicted = 3.60, actual = 4.00
  x = 4, predicted = 4.45, actual = 4.50
  x = 5, predicted = 5.30, actual = 5.00


Bayesian Linear Regression

Unlike standard linear regression, Bayesian regression treats the weights as random variables with a prior distribution. We combine a Gaussian prior with the data likelihood to get a posterior over weights. In practice, taking a normal prior $p(w)\propto \exp(-\frac{1}{2\sigma^2}|w|^2)$ leads to a regularized (Ridge) regression: one adds a penalty $\lambda|w|^2$ to the loss. Fitting then becomes finding a posterior mean of $w$. Conceptually:
posterior ∝ exp⁡(−12σ2∥w∥2)⋅exp⁡(−N⋅MSE(w,b)/2).posterior ∝ exp(−2σ21​∥w∥2)⋅exp(−N⋅MSE(w,b)/2).
The update rule adds $\lambda w$ to the gradient (MAP estimate). This gives the same effect as Ridge regression. Bayesian inference can also yield uncertainty estimates for $w$, but those require sampling (beyond basic NumPy).



Summary:
By placing a zero‑mean Gaussian prior on the weights
p(w)∝exp⁡(−12σ2∥w∥2),
p(w)∝exp(−2σ21​∥w∥2),

and combining it with the usual Gaussian likelihood of the data, the posterior becomes
p(w∣X,y)  ∝  exp⁡(−12σ2∥w∥2) exp⁡(−N2 MSE(w,b)).
p(w∣X,y)∝exp(−2σ21​∥w∥2)exp(−2N​MSE(w,b)).

Finding the MAP estimate of this posterior is equivalent to minimizing
MSE(w,b)+λ∥w∥2MSE(w,b)+λ∥w∥2,
i.e. Ridge regression with penalty λ∝1/σ2λ∝1/σ2.
In gradient descent, this adds a term λ wλw to the weight‐gradient, giving the update
w  ←  w−η[∇w MSE(w,b)+λ w].
w←w−η[∇w​MSE(w,b)+λw].

Below we outline each step with citations, then show the full annotated Python file.
Bayesian Linear Regression: Gaussian Prior & Posterior
Gaussian Prior on Weights

In Bayesian regression we treat the weight vector ww as a random variable with a Gaussian prior
p(w)  ∝  exp⁡(−12σ2∥w∥2).
p(w)∝exp(−2σ21​∥w∥2).

This encodes our belief that large weights are unlikely
Let’s talk about science!
and is exactly the same as the penalty term in Ridge regression
Medium
.
Likelihood & Posterior

Assuming Gaussian noise on observations gives a likelihood
∝exp⁡(−N2 MSE(w,b))∝exp(−2N​MSE(w,b)).
Multiplying prior and likelihood yields the posterior
p(w∣X,y)  ∝  exp⁡(−12σ2∥w∥2) exp⁡(−N2 MSE(w,b)).
p(w∣X,y)∝exp(−2σ21​∥w∥2)exp(−2N​MSE(w,b)).

Taking the negative log gives the familiar Ridge objective
Computer Science at Princeton
.
MAP Estimate & Ridge Equivalence
Loss with L2 Penalty

The MAP estimate minimizes
1N∑i(yi−y^i)2⏟MSE  +  λ ∥w∥2,
MSE
N1​i∑​(yi​−y^​i​)2​​+λ∥w∥2,

where λ=12σ2λ=2σ21​
Statistical Odds & Ends
.
This is the Ridge regression cost function
Cross Validated
.
Gradient‐Descent Update

For standard MSE,
∇w MSE=1NXT(Xw+b−y)∇w​MSE=N1​XT(Xw+b−y).
Adding the prior gives
∇w [MSE+λ∥w∥2]=1NXT(Xw+b−y)  +  λ w.
∇w​[MSE+λ∥w∥2]=N1​XT(Xw+b−y)+λw.

Thus the weight update becomes
w←w  −  η[1NXT(Xw+b−y)+λ w]
w←w−η[N1​XT(Xw+b−y)+λw]


In [ ]:
#!/usr/bin/env python3
# bayesian_linear_regression.py

"""
Bayesian (MAP) Linear Regression via Gradient Descent
-----------------------------------------------------
This script fits a line y = w*x + b using MAP estimation,
which with a Gaussian prior on w is equivalent to Ridge regression.

Key additions over plain linear regression:
- A Gaussian prior p(w) ∝ exp(-||w||^2 / (2σ^2)) yields penalty λ||w||^2.
- Gradient adds λ * w when updating weights.
"""

import numpy as np

def main():
    # -----------------------------
    # 1. Create example data
    # -----------------------------
    X = np.array([[1], [2], [3], [4], [5]])  # hours practiced
    y = np.array([1.5, 3.0, 4.0, 4.5, 5.0])  # points scored

    # -----------------------------
    # 2. Initialize parameters
    # -----------------------------
    w = np.random.rand(1)   # initial weight
    b = 0.0                 # initial bias

    # Hyperparameters
    lr      = 0.1           # learning rate
    n_iters = 1000          # training iterations
    N       = len(y)        # number of samples

    # Bayesian prior strength → Ridge penalty
    lambda_ = 0.5           # regularization coefficient

    # -----------------------------
    # 3. Training loop
    # -----------------------------
    for i in range(n_iters):
        # Forward pass: predictions
        y_pred = X.dot(w) + b

        # Compute basic MSE gradients
        error = y_pred - y                   # shape (N,)
        dw_basic = (1/N) * X.T.dot(error)    # shape (1,)
        db      = (1/N) * np.sum(error)      # scalar

        # Add prior (Ridge) gradient: λ * w
        dw = dw_basic + lambda_ * w

        # Update parameters
        w -= lr * dw
        b -= lr * db

    # -----------------------------
    # 4. Results
    # -----------------------------
    print(f"MAP-estimated weight: {w[0]:.4f}, bias: {b:.4f}\n")

    print("Final predictions vs actual:")
    for x_val, actual in zip(X.flatten(), y):
        pred = w * x_val + b
        print(f"  x={x_val}, pred={pred[0]:.2f}, actual={actual:.2f}")

if __name__ == "__main__":
    main()


MAP-estimated weight: 0.6800, bias: 1.5600

Final predictions vs actual:
  x=1, pred=2.24, actual=1.50
  x=2, pred=2.92, actual=3.00
  x=3, pred=3.60, actual=4.00
  x=4, pred=4.28, actual=4.50
  x=5, pred=4.96, actual=5.00


# Hierarchical Time Series Regression
What Is Hierarchical Time‑Series Regression?

Hierarchical time‑series regression extends standard regression by introducing a multi‑level parameter structure:

    Global level: Defines hyperpriors (e.g., overall mean intercept and slope) shared across all series.

    Group level: Specifies per‑series parameters (e.g., each store’s intercept and trend), drawn from the global distributions.
    Wikipedia

This setup contrasts with:

    Complete pooling, where one regression fits all data, ignoring series differences.

    No pooling, where each series is modeled separately, risking overfitting when data are sparse.
    arXiv

Key Benefits of the Hierarchical Structure

    Partial Pooling / Borrowing Strength
    By sharing information through hyperpriors, series with few observations “borrow” data from the whole ensemble, reducing variance in parameter estimates
    Your Name Here's Homepage
    .

    Adaptive Regularization
    The degree of pooling is learned from the data: when series are similar, the model pools more; when they differ, it pools less
    otexts.com
    .

    Improved Forecast Coherence
    In strictly hierarchical hierarchies (e.g., country → region → store), forecasts are internally consistent: the sum of lower‑level forecasts matches higher‑level totals
    CRAN
    .

    Uncertainty Quantification
    Full Bayesian inference yields credible intervals for both global and series‑specific parameters, offering richer insights than point estimates alone
    leg.ufpr.br
    .

Mathematical Foundations

Let there be JJ series, each with data {(tj,i,yj,i)}i=1Nj{(tj,i​,yj,i​)}i=1Nj​​. A simple hierarchical linear model is:

    Hyperpriors (Global Level)
    μa∼N(0, σa2),μb∼N(0, σb2)
    μa​∼N(0,σa2​),μb​∼N(0,σb2​)

    Group-Level Priors (Per-Series)
    aj∼N(μa, σa2),bj∼N(μb, σb2)
    aj​∼N(μa​,σa2​),bj​∼N(μb​,σb2​)

    Observation Model (Likelihood)
    yj,i∼N(aj+bj tj,i,  σobs2)
    yj,i​∼N(aj​+bj​tj,i​,σobs2​)

    where σobs2σobs2​ captures noise in observations.
    Wikipedia

The posterior is then sampled (e.g., via MCMC) or approximated (e.g., variational inference) to infer all parameters jointly
leg.ufpr.br
.
Practical Applications & Tools

    Retail forecasting: Model sales for multiple stores or regions, ensuring aggregate forecasts match corporate totals while capturing individual store trends
    otexts.com
    .

    Sensor networks: Pool data across sensors in different locations to improve calibration and detect anomalies
    Reddit
    .

    Hierarchical reconciliation: Use methods like bottom‑up or optimal combination (MinT) to reconcile forecasts across aggregation levels, implemented in R’s hts and htsDegenerateR packages
    CRAN
    .

    Bayesian frameworks:

        PyMC: Structure hierarchical models with intuitive context managers and NUTS sampling
        PyMC Discourse
        .

        Stan / NumPyro: Similar hierarchical syntax supporting both MCMC and variational inference.


In [ ]:
#!/usr/bin/env python3
# hierarchical_time_series_regression.py

import numpy as np
import pymc as pm
import arviz as az

def simulate_data(n_groups=3, n_time=10, sigma=0.5, seed=42):
    """Simulate n_groups time series of length n_time with
       group-specific intercepts and slopes."""
    rng = np.random.default_rng(seed)
    time = np.arange(n_time)
    true_a = np.array([1.0, 2.0, 3.0])           # intercepts
    true_b = np.array([0.5, -0.2, 0.1])          # slopes
    y = np.zeros((n_groups, n_time))
    for j in range(n_groups):
        y[j] = true_a[j] + true_b[j] * time + rng.normal(0, sigma, size=n_time)
    return time, y

def fit_hierarchical_model(time, y):
    n_groups, n_time = y.shape

    with pm.Model() as model:
        # Hyperpriors for intercepts and slopes
        mu_a    = pm.Normal("mu_a", mu=0, sigma=10)
        sigma_a = pm.HalfNormal("sigma_a", sigma=1)
        mu_b    = pm.Normal("mu_b", mu=0, sigma=1)
        sigma_b = pm.HalfNormal("sigma_b", sigma=1)

        # Group‑level (partial pooling)
        a = pm.Normal("a", mu=mu_a, sigma=sigma_a, shape=n_groups)
        b = pm.Normal("b", mu=mu_b, sigma=sigma_b, shape=n_groups)

        # Observation noise
        sigma_obs = pm.HalfNormal("sigma_obs", sigma=1)

        # Expected value per group and time
        y_est = a[:, None] + b[:, None] * time

        # Likelihood
        pm.Normal("y_obs", mu=y_est, sigma=sigma_obs, observed=y)

        # Inference
        trace = pm.sample(1000, tune=1000, target_accept=0.9, cores=2)
    return model, trace

def main():
    # 1) Simulate data
    time, y = simulate_data()
    # 2) Fit model
    model, trace = fit_hierarchical_model(time, y)
    # 3) Summarize
    # print(az.summary(trace, var_names=["mu_a","sigma_a","mu_b","sigma_b","sigma_obs"]))

if __name__ == "__main__":
    main()


Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu_a, sigma_a, mu_b, sigma_b, a, b, sigma_obs]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Output()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# check if gpu is available
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)
    
if __name__ == "__main__":
    model = LinearRegression(1, 1)
    print(model)
    print(model.parameters())
    print(model.state_dict())
    print(model.forward(torch.tensor([1.0])))
    print(model.forward(torch.tensor([2.0])))
    print(model.forward(torch.tensor([3.0])))
    print(model.forward(torch.tensor([4.0])))
    print(model.forward(torch.tensor([5.0])))

True
NVIDIA GeForce RTX 5080 Laptop GPU
LinearRegression(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)
<generator object Module.parameters at 0x75f06390e110>
OrderedDict([('linear.weight', tensor([[-0.9560]])), ('linear.bias', tensor([0.1967]))])
tensor([-0.7592], grad_fn=<ViewBackward0>)
tensor([-1.7152], grad_fn=<ViewBackward0>)
tensor([-2.6711], grad_fn=<ViewBackward0>)
tensor([-3.6271], grad_fn=<ViewBackward0>)
tensor([-4.5830], grad_fn=<ViewBackward0>)


# Classification Algorithms

Classification predicts categories (e.g. win/lose, or player position). We cover several common classifiers.
Logistic Regression

Logistic regression models the probability of a binary outcome (0/1). It uses the sigmoid (logistic) function to squeeze a linear score into [0,1]:
y^=f(x)=11+exp⁡(−η(x)),η(x)=wTx+b,y^​=f(x)=1+exp(−η(x))1​,η(x)=wTx+b,
so $f(x)\approx P(Y=1|x)$
realpython.com
. Here $\eta(x)$ is called the log-odds (logit). The sigmoid ensures outputs near 0 or 1, suitable for classification. For example, to predict if a team wins (1) or loses (0) based on stats: we compute $\eta = w\cdot x+b$ and then probability $\hat y = \frac1{1+e^{-\eta}}$
realpython.com
.

To train logistic regression, we maximize the (log-)likelihood of the binary labels or equivalently minimize binary cross-entropy loss:
L(w,b)=−1N∑i=1N[yilog⁡(y^i)+(1−yi)log⁡(1−y^i)].L(w,b)=−N1​∑i=1N​[yi​log(y^​i​)+(1−yi​)log(1−y^​i​)].
Gradient descent updates are similar to linear regression but use the sigmoid derivative. The weight gradient is
∇wL=1NXT(y^−y),∇w​L=N1​XT(y^​−y),
and $b$-gradient is $\frac{1}{N}\sum (\hat y - y)$. Implementation example with a small dataset (e.g. predicts win=1 if team score exceeds opponent’s by a threshold):

In [ ]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def compute_loss(y, y_pred, w, reg_lambda):
    # binary cross-entropy + L2 penalty
    ce = -np.mean(y*np.log(y_pred) + (1-y)*np.log(1-y_pred))
    reg = (reg_lambda / (2*len(y))) * np.sum(w**2)
    return ce + reg

def train_logistic(
    X, y,
    lr=0.01,
    n_iters=1000,
    batch_size=None,
    reg_lambda=0.0,
    momentum=0.0,
    verbose=False
):
    N, m = X.shape
    # 1) Initialize
    w = np.random.randn(m) * 0.01
    b = 0.0
    v_w = np.zeros_like(w)
    v_b = 0.0

    for epoch in range(n_iters):
        # 2) Optionally shuffle for SGD/mini-batch
        idx = np.random.permutation(N)
        X_sh, y_sh = X[idx], y[idx]

        # 3) Determine batches
        batches = (
            [ (X_sh, y_sh) ] if batch_size is None
            else [
                (X_sh[i:i+batch_size], y_sh[i:i+batch_size])
                for i in range(0, N, batch_size)
            ]
        )

        for X_batch, y_batch in batches:
            # 4a) Forward pass
            z = X_batch.dot(w) + b
            y_pred = sigmoid(z)

            # 4b) Gradients w/ L2
            dw = (X_batch.T.dot(y_pred - y_batch) / len(y_batch)
                  + (reg_lambda/len(y_batch))*w)
            db = np.mean(y_pred - y_batch)

            # 4c) Momentum update
            v_w = momentum*v_w + lr*dw
            v_b = momentum*v_b + lr*db
            w -= v_w
            b -= v_b

        if verbose and epoch % (n_iters//10 or 1) == 0:
            loss = compute_loss(y, sigmoid(X.dot(w)+b), w, reg_lambda)
            print(f"Epoch {epoch}: loss={loss:.4f}")

    return w, b

# Example usage:
X = np.array([[50],[55],[60],[65],[70]])
y = np.array([0,0,1,1,1])
# Feature‐scale
X = (X - X.mean()) / X.std()

w, b = train_logistic(
    X, y,
    lr=0.05,
    n_iters=500,
    batch_size=2,       # mini‐batch of 2
    reg_lambda=0.1,     # L2 strength
    momentum=0.9,
    verbose=True
)
print("Final w,b:", w, b)


Epoch 0: loss=0.6447
Epoch 50: loss=0.2672
Epoch 100: loss=0.2644
Epoch 150: loss=0.2697
Epoch 200: loss=0.2683
Epoch 250: loss=0.2665
Epoch 300: loss=0.2749
Epoch 350: loss=0.2641
Epoch 400: loss=0.2662
Epoch 450: loss=0.2684
Final w,b: [1.77348886] 0.7236525850267588


Bayesian Logistic Regression

Bayesian logistic regression treats weights as random with a prior. Unlike “classical” logistic (which finds a single best $w$ by maximum likelihood), Bayesian methods infer a posterior distribution of $w$. In simple terms, we start with a prior $p(w)$ and update via Bayes’ rule:
p(w∣X,y)∝p(y∣X,w) p(w).p(w∣X,y)∝p(y∣X,w)p(w).
This means the posterior ∝ (likelihood × prior)
stats.stackexchange.com
. For example, with a normal prior on $w$, the MAP estimate adds an $L_2$ penalty to the loss. We can approximate the posterior by sampling or variational methods (beyond basic NumPy). In practice, Bayesian logistic allows us to quantify uncertainty in predictions.

In [ ]:
"""
bayes_logreg.py

A from‑scratch, step‑by‑step guide to Bayesian logistic regression.
Includes:
  • Data setup
  • MAP estimation (L2 ‑ ridge)
  • Laplace approximation around the MAP
  • Full posterior sampling with PyMC (NUTS)
  • Posterior predictive checks
Adjustable parameters and notes indicated via comments throughout.
"""

# 0) Imports
import numpy as np
from scipy.optimize import minimize
import pymc as pm
import arviz as az
import math

# 1) Data setup
# ------------------------
# Binary outcome y = win (1) / loss (0)
X_raw = np.array([50, 55, 60, 65, 70])[:, None]
y     = np.array([0,   0,  1,  1,  1])
# Standardize features for stability
X = (X_raw - X_raw.mean()) / X_raw.std()
N, m = X.shape

# 2) Helper functions
# ------------------------
def sigmoid(z):
    """Standard logistic sigmoid."""
    return 1.0 / (1.0 + np.exp(-z))

def neg_log_post(theta, X, y, sigma2=1.0):
    """
    Negative log‑posterior for MAP:
      theta = [w_0, ..., w_{m-1}, b]
      prior: w ~ N(0, sigma2 * I)
    Adjust sigma2 ↓ for stronger shrinkage.
    """
    w, b = theta[:-1], theta[-1]
    z = X.dot(w) + b
    ll = np.sum(y * np.log(sigmoid(z)) + (1 - y) * np.log(1 - sigmoid(z)))
    # log‑prior ∝ - 0.5 * w^T w / sigma2
    log_prior = -0.5 * np.sum(w**2) / sigma2
    return -(ll + log_prior)   # we minimize this

# 3) MAP estimation
# ------------------------
# Initial guess: zero weights & bias
theta0 = np.zeros(m + 1)
# Run BFGS to find MAP
opt = minimize(
    neg_log_post, theta0,
    args=(X, y, 1.0),       # tweak sigma2 here (e.g. 0.5 → stronger prior)
    method='BFGS',
    options={'gtol': 1e-6, 'maxiter': 1000}
)
w_map, b_map = opt.x[:-1], opt.x[-1]
print("=== MAP ESTIMATE ===")
print("Weights:", w_map)
print("Bias:   ", b_map)

# 4) Laplace approximation
# ------------------------
# Approximate posterior covariance ≈ inverse Hessian at MAP
hess_inv = opt.hess_inv      # SciPy BFGS gives you this
cov_map  = hess_inv           # shape (m+1, m+1)
print("\n=== LAPLACE APPROXIMATION ===")
print("Posterior SDs:", np.sqrt(np.diag(cov_map)))

# To draw from Laplace‑Gaussian:
# theta_samples = np.random.multivariate_normal(opt.x, cov_map, size=5000)

# 5) Full Bayesian inference with PyMC (NUTS)
# ------------------------
print("\n=== FULL BAYES WITH PYMC ===")
with pm.Model() as bayes_logreg:
    # priors: Normal(0, 1) for each weight & bias; adjust sd for stronger/weaker shrinkage
    w = pm.Normal('w', mu=0, sigma=1, shape=m)
    b = pm.Normal('b', mu=0, sigma=1)

    # linear predictor and likelihood
    logits = pm.math.dot(X, w) + b
    pm.Bernoulli('obs', logit_p=logits, observed=y)

    # Inference: NUTS sampler
    trace = pm.sample(
        draws=2000,       # post‑tuning samples per chain
        tune=1000,        # burn‑in
        target_accept=0.9,
        chains=2,
        return_inferencedata=True
    )

# Summarize posterior
print(az.summary(trace, var_names=['w', 'b'], hdi_prob=0.95))

# 6) Posterior‑predictive probabilities (robust fix)
# --------------------------------------------------
with bayes_logreg:
    ppc = pm.sample_posterior_predictive(
        trace,
        var_names=['obs'],
        return_inferencedata=False
    )

# Inspect the raw shape to guide our averaging
print("DEBUG: ppc['obs'] shape =", ppc['obs'].shape)

# Collapse any extra sample dims into one, then average
samples = ppc['obs']
if samples.ndim > 2:
    samples = samples.reshape(-1, samples.shape[-1])
pp_win_probs = samples.mean(axis=0)

# Print nicely by converting each to Python scalars
print("\n=== POSTERIOR‑PREDICTIVE WIN PROBS ===")
for xi, p in zip(X_raw.flatten(), pp_win_probs):
    print(f"Rating {int(xi):>2} → P(win) ≈ {float(p):.2f}")



# 7) Where to adjust next
# ------------------------
"""
• Stronger “regularization”: 
    - In neg_log_post: set sigma2 < 1.0
    - In PyMC priors: use pm.Normal(..., sigma=<0.5 or less>)
• Switch to L1 (sparse) prior:
    - In PyMC: replace w = pm.Normal(...) with pm.Laplace('w', mu=0, b=<scale>)
• Faster approx:
    - Use Laplace only; draw samples from theta ~ N(opt.x, cov_map)
    - Swap in ADVI: pm.fit(method='advi', ...) instead of pm.sample()
• Scale to large data:
    - In pm.fit: use method='advi_minibatch', minibatch_size=..., n → millions
• Partial pooling / hierarchical:
    - Replace w ~ Normal(μ, τ) and add hyperpriors:
          μ_w = pm.Normal('μ_w', 0, 1)
          τ_w = pm.HalfNormal('τ_w', 1)
          w   = pm.Normal('w', μ=μ_w, sigma=τ_w, shape=m)
• Feature interactions:
    - Expand X: e.g. X_poly = np.hstack([X, X**2, ...])
• Diagnostics:
    - Check az.plot_trace(trace)
    - Compute R̂ via az.rhat(trace)
    - Compare WAIC / LOO: az.waic(trace), az.loo(trace)
"""

# If you like, add final plotting or saving of results here.


=== MAP ESTIMATE ===
Weights: [1.03697116]
Bias:    0.5087098425833299

=== LAPLACE APPROXIMATION ===
Posterior SDs: [0.7468874  1.01527945]

=== FULL BAYES WITH PYMC ===


Initializing NUTS using jitter+adapt_diag...
/app/.venv/lib/python3.10/site-packages/pytensor/link/c/cmodule.py:2968: UserWarning: PyTensor could not link to a BLAS installation. Operations that might benefit from BLAS will be severely degraded.
This usually happens when PyTensor is installed via pip. We recommend it be installed via conda/mamba/pixi instead.
Alternatively, you can use an experimental backend such as Numba or JAX that perform their own BLAS optimizations, by setting `pytensor.config.mode == 'NUMBA'` or passing `mode='NUMBA'` when compiling a PyTensor function.
For more options and details see https://pytensor.readthedocs.io/en/latest/troubleshooting.html#how-do-i-configure-test-my-blas-library
  warnings.warn(
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [w, b]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatibl

Output()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 1 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


       mean     sd  hdi_2.5%  hdi_97.5%  mcse_mean  mcse_sd  ess_bulk  \
w[0]  1.129  0.766    -0.343      2.697      0.013    0.012    3265.0   
b     0.275  0.730    -1.190      1.647      0.013    0.011    3242.0   

      ess_tail  r_hat  
w[0]    2687.0    1.0  
b       2723.0    1.0  


Sampling: [obs]


Output()

DEBUG: ppc['obs'] shape = (2, 2000, 5)

=== POSTERIOR‑PREDICTIVE WIN PROBS ===
Rating 50 → P(win) ≈ 0.27
Rating 55 → P(win) ≈ 0.39
Rating 60 → P(win) ≈ 0.57
Rating 65 → P(win) ≈ 0.71
Rating 70 → P(win) ≈ 0.81


"\n• Stronger “regularization”: \n    - In neg_log_post: set sigma2 < 1.0\n    - In PyMC priors: use pm.Normal(..., sigma=<0.5 or less>)\n• Switch to L1 (sparse) prior:\n    - In PyMC: replace w = pm.Normal(...) with pm.Laplace('w', mu=0, b=<scale>)\n• Faster approx:\n    - Use Laplace only; draw samples from theta ~ N(opt.x, cov_map)\n    - Swap in ADVI: pm.fit(method='advi', ...) instead of pm.sample()\n• Scale to large data:\n    - In pm.fit: use method='advi_minibatch', minibatch_size=..., n → millions\n• Partial pooling / hierarchical:\n    - Replace w ~ Normal(μ, τ) and add hyperpriors:\n          μ_w = pm.Normal('μ_w', 0, 1)\n          τ_w = pm.HalfNormal('τ_w', 1)\n          w   = pm.Normal('w', μ=μ_w, sigma=τ_w, shape=m)\n• Feature interactions:\n    - Expand X: e.g. X_poly = np.hstack([X, X**2, ...])\n• Diagnostics:\n    - Check az.plot_trace(trace)\n    - Compute R̂ via az.rhat(trace)\n    - Compare WAIC / LOO: az.waic(trace), az.loo(trace)\n"

K-Nearest Neighbors (KNN)

KNN is a simple non-parametric classifier (or regressor) based on distance. Given a query point, it finds the k closest training samples (by Euclidean distance) and uses their labels. For classification, KNN predicts by majority vote of neighbors; for regression, by averaging neighbors
digitalocean.com
. For example, to classify a player as “good” or “average” (0/1), we might look at the three nearest players in feature space and choose the majority label.

An implementation outline:

In [ ]:
import math
from collections import Counter

# Synthetic dataset: [height (cm), weight (kg)] -> Position (F=Forward, G=Guard)
players = np.array([
    [198, 95, 'F'],   # a taller, heavier Forward
    [202, 100, 'F'], 
    [180, 78, 'G'],   # a shorter, lighter Guard
    [175, 68, 'G'],
    [188, 85, 'F'],
    [190, 90, 'F'],
    [170, 72, 'G'],
    [185, 80, 'F']
])
X_train = players[:, :2].astype(float)
y_train = players[:, 2]

def euclidean_distance(p1, p2):
    return math.sqrt(np.sum((p1 - p2)**2))

def knn_predict(x_query, X_train, y_train, k):
    # Compute distances from x_query to all training points
    distances = [euclidean_distance(x_query, x) for x in X_train]
    # Get the indices of the k smallest distances
    nn_idx = np.argsort(distances)[:k]
    nn_labels = y_train[nn_idx]
    # Majority vote
    vote = Counter(nn_labels).most_common(1)[0][0]
    return vote

# Predict position for a new player of height=192cm, weight=88kg using k=3
new_player = np.array([192.0, 88.0])
pred_position = knn_predict(new_player, X_train, y_train, k=3)
print(f"Predicted position for player {new_player}: {pred_position}")


Predicted position for player [192.  88.]: F


Decision Trees
Theory – Splitting Criteria and Tree Construction

A Decision Tree is a flowchart-like tree structure where each internal node represents a decision on a feature, each branch is an outcome of that decision, and each leaf node represents a prediction (class label or numeric value). They can handle both classification and regression tasks and are valued for their interpretability (rules are easy to explain)
medium.com
.

How it works: The tree is built by recursively splitting the data based on features. The algorithm (e.g., CART – Classification and Regression Tree) searches for the feature and threshold that best purifies the data at each step. Purity is measured by metrics like:

    Gini Impurity: G=1−∑jpj2,G=1−∑j​pj2​, where pjpj​ is the fraction of examples of class j in a node. Gini is 0 for a perfectly pure node (all examples same class) and highest when classes are evenly mixed
    quantdare.com
    .

    Entropy: H=−∑jpjlog⁡2pj,H=−∑j​pj​log2​pj​, another measure of impurity from information theory
    quantdare.com
    . Entropy is 0 when pure and maximal when classes are equally mixed (e.g., 50/50 gives entropy = 1 bit).

    Information Gain: The decrease in entropy from a split. If a node has entropy H before splitting and the weighted entropy of children after splitting is H<sub>split</sub>, then Information Gain = H - H<sub>split</sub>
    en.wikipedia.org
    en.wikipedia.org
    . The algorithm seeks the split that maximizes this gain (or equivalently, maximizes Gini reduction).

For regression trees, a common criterion is to minimize the variance (MSE) within each partition.

Splitting process: Starting at the root (all data), the tree chooses the best feature and threshold to split on (e.g., “if assists > 5 go left, else go right”), based on highest information gain
en.wikipedia.org
en.wikipedia.org
. This creates two child nodes. The process repeats recursively: at each non-leaf node, evaluate all possible splits to find the best one. Splitting stops when a stopping criterion is met, e.g., maximum tree depth, minimum samples in a node, or no further impurity reduction (perfectly pure or all features used).

Leaf predictions: In classification, a leaf is assigned a class (often the majority class of training examples in that leaf). In regression, the leaf predicts a numeric value (often the mean of targets in that leaf).

Avoiding overfitting: Unrestricted trees can grow very deep and complex, perfectly fitting training data (zero impurity leaves), which often overfits. To prevent this, we use pre-pruning (stop splits early if data gets too sparse or depth too high) or post-pruning (grow a full tree, then prune back leaves that don’t generalize well via validation).

In [ ]:
import numpy as np

# Example sports game data: [home_court, star_player_points] -> win(1)/lose(0)
# home_court: 1 if playing at home, 0 if away
data = np.array([
    [1, 30, 1],  # home, star scored 30 -> win
    [1, 15, 0],  # home, star scored 15 -> lose
    [0, 25, 0],  # away, 25 -> lose
    [0, 28, 0],  # away, 28 -> lose
    [1, 8,  0],  # home, only 8 points -> lose
    [0, 35, 1],  # away, 35 -> win (away win unusual but star played exceptionally)
    [1, 20, 1],  # home, 20 -> win
    [0, 10, 0]   # away, 10 -> lose
])
X = data[:, :2]
y = data[:, 2]

# Function to compute Gini impurity of a label array
def gini_impurity(labels):
    if len(labels) == 0:
        return 0
    p = np.mean(labels)  # proportion of class 1
    q = 1 - p            # proportion of class 0
    return 1 - (p**2 + q**2)

# Try all possible splits and find best (brute-force for demonstration)
best_gini = 1.0
best_rule = None
for feature in [0, 1]:
    values = np.unique(X[:, feature])
    for val in values:
        # For numeric feature, consider splitting <= val vs > val 
        left_idx = (X[:, feature] <= val)
        right_idx = (X[:, feature] > val)
        gini_left = gini_impurity(y[left_idx])
        gini_right = gini_impurity(y[right_idx])
        # weighted gini after split
        n_left, n_right = np.sum(left_idx), np.sum(right_idx)
        gini_split = (n_left * gini_left + n_right * gini_right) / len(y)
        if gini_split < best_gini:
            best_gini = gini_split
            best_rule = (feature, val, gini_split, gini_left, gini_right)

print("Best split:", best_rule)


Best split: (1, 28, 0.20833333333333326, 0.2777777777777777, 0.0)


Random Forests
Theory – Ensemble of Decision Trees

A Random Forest is an ensemble method that combines multiple decision trees to improve robustness and accuracy
medium.com
. The core idea is to build many decision trees on random subsets of the data and features, and then aggregate their predictions. It’s like getting a “wisdom of the crowd” from many tree models.

Key elements of Random Forests:

    Bootstrap Aggregating (Bagging): From the training set of size n, sample B new datasets of size n with replacement (each tree gets a bootstrap sample)
    en.wikipedia.org
    . Each tree is trained on its bootstrap sample. This introduces diversity among trees (each tree sees a slightly different dataset).

    Random Feature Subset: When splitting a node, instead of considering all features, Random Forests pick a random subset of features and find the best split among those
    en.wikipedia.org
    en.wikipedia.org
    . For classification, a common default is dd

    ​ features (if there are d total features)
    en.wikipedia.org
    . This further decorrelates the trees, so they don’t all choose the same dominant feature.

    Tree independence: Each tree is grown to full depth (or with minimal pruning) on its bootstrap data and feature subsets. The trees will overfit individually, but since they’re all different, their errors can cancel out when averaged.

    Aggregation: For classification, the forest takes a majority vote among the trees’ predicted classes. For regression, it averages the predicted values of all trees
    en.wikipedia.org
    . This averaging significantly reduces variance compared to a single tree
    en.wikipedia.org
    en.wikipedia.org
    , resulting in a more stable and accurate model.

The random forest algorithm can be summarized as
en.wikipedia.org
en.wikipedia.org
:

    For b = 1 to B (number of trees):

        Sample n examples from training data with replacement (bootstrap sample).

        Train a decision tree TbTb​ on this sample. At each split, consider a random subset of features (of size k) instead of all d features, choose the best split among those.

    To predict a new example:

        Get the prediction from each tree Tb(x)Tb​(x).

        Classification: output the majority-vote class. Regression: output the average 1B∑b=1BTb(x)B1​∑b=1B​Tb​(x)
        en.wikipedia.org
        .

The combination of bagging and feature randomness makes Random Forests resistant to overfitting; even though each tree might overfit, their average can still generalize well
en.wikipedia.org
. They also provide useful features like out-of-bag (OOB) error (each training instance can be predicted by trees that didn’t see it in training, giving an internal validation estimate)
en.wikipedia.org
en.wikipedia.org
and feature importance (measuring how much each feature split improved the purity on average).

In [ ]:
import math

# Let's manually create 3 bootstrap samples from our previous data
np.random.seed(1)
B = 3
n = len(X)
boot_samples = [data[np.random.choice(n, n, replace=True)] for _ in range(B)]

# Train a simple stump (1-level tree) on each bootstrap by splitting on HomeCourt
predictions = []
for b_idx, sample in enumerate(boot_samples):
    X_b, y_b = sample[:, :2], sample[:, 2]
    # Train a stump: we'll use the best split we found before (HomeCourt <= 0)
    # Determine majority class for home=0 and home=1 in this sample
    home_mask = (X_b[:,0] > 0)
    pred_home = 1 if np.mean(y_b[home_mask]) >= 0.5 else 0
    pred_away = 1 if np.mean(y_b[~home_mask]) >= 0.5 else 0
    # Define a simple prediction function for this stump
    def tree_predict(instance):
        return pred_home if instance[0] > 0 else pred_away
    # Record predictions of this tree on original data
    preds = [tree_predict(xi) for xi in X]
    predictions.append(preds)
    print(f"Tree {b_idx+1} prediction on all games: {preds}")

# Combine predictions by majority vote
predictions = np.array(predictions)  # shape (B, n)
final_preds = (np.mean(predictions, axis=0) >= 0.5).astype(int)
print("Random Forest combined prediction:", final_preds.tolist())
print("Actual outcomes:               ", y.astype(int).tolist())


Tree 1 prediction on all games: [0, 0, 0, 0, 0, 0, 0, 0]
Tree 2 prediction on all games: [0, 0, 0, 0, 0, 0, 0, 0]
Tree 3 prediction on all games: [0, 0, 0, 0, 0, 0, 0, 0]
Random Forest combined prediction: [0, 0, 0, 0, 0, 0, 0, 0]
Actual outcomes:                [1, 0, 0, 0, 0, 1, 1, 0]


Support Vector Machines (SVM)

Support Vector Machines classify data by finding the hyperplane that maximizes the margin between classes
analyticsvidhya.com
analyticsvidhya.com
. The margin is the distance from the hyperplane to the nearest data points (support vectors). Formally, SVM solves:
min⁡w,b 12∥w∥2s.t.  yi(wTxi+b)≥1 ∀i,minw,b​ 21​∥w∥2s.t.yi​(wTxi​+b)≥1 ∀i,
seeking the widest margin. The result is a decision boundary that separates classes as much as possible. In practice, soft margins and kernels allow handling noisy and non-linear data. SVM predictions are $\text{sign}(w^T x + b)$.

    Margin maximization: “The goal is to maximize the minimum distance” between any training point and the decision boundary
    analyticsvidhya.com
    .

    Non-linear data: Kernels map data to higher dimensions so that a linear hyperplane can separate them. SVM is powerful for complex boundaries
    analyticsvidhya.com
    .

A simple linear SVM can be implemented by minimizing the hinge loss $L_i=\max(0,1-y_i(w^T x_i+b))$. Gradient descent would add updates only for misclassified or “margin-violating” points. (Detailed code is lengthy, but conceptually one iterates: if $y_i(w^T x_i+b)<1$, update $w$ by $w \leftarrow w + \eta y_i x_i$, otherwise apply regularization.) The key outcome is a robust classifier with maximum margin
analyticsvidhya.com
analyticsvidhya.com
.

In [ ]:
# Synthetic data: [offense_rating, defense_rating] -> win (+1) or lose (-1)
X = np.array([
    [8, 5],   # strong offense, moderate defense -> win
    [3, 7],   # weak offense, strong defense -> (assume) lose
    [6, 6],   # balanced -> win
    [4, 4],   # weak both -> lose
])
y = np.array([1, -1, 1, -1])  # +1 = win, -1 = lose

# Initialize weights and bias
w = np.zeros(X.shape[1])
b = 0.0
C = 1.0      # regularization parameter
alpha = 0.1  # learning rate

# Training using sub-gradient descent for hinge loss
for epoch in range(1000):
    for i in range(len(X)):
        if y[i] * (np.dot(w, X[i]) + b) < 1:
            # misclassified or within margin
            w = w - alpha * (w - C * y[i] * X[i])  # sub-gradient: w term + hinge term
            b = b + alpha * C * y[i]               # update for bias
        else:
            # correctly classified and outside margin
            w = w - alpha * w                      # only regularization term
            # b update has no hinge term when correctly classified

# Results
print("Learned weight vector:", w)
print("Learned bias:", b)
# Predict on a new example
new_game = np.array([5, 7])
score = np.dot(w, new_game) + b
pred_label = 1 if score >= 0 else -1
print("Decision function output for new game:", score)
print("Predicted outcome:", "Win" if pred_label==1 else "Lose")


Learned weight vector: [0.64964788 0.26271842]
Learned bias: -4.6
Decision function output for new game: 0.48726835737247143
Predicted outcome: Win


Naive Bayes
Theory – Bayes’ Theorem with Independence Assumption

Naive Bayes is a probabilistic classifier based on applying Bayes’ Theorem with a strong assumption: all features are conditionally independent given the class
geeksforgeeks.org
. Despite this “naive” assumption (which is often false in reality), the classifier performs surprisingly well in many domains, especially text classification.

Bayes’ Theorem reminds us that for class CC and features X=(x1,…,xn)X=(x1​,…,xn​):
P(C∣X)=P(X∣C) P(C)P(X).P(C∣X)=P(X)P(X∣C)P(C)​.

For classification, we compare P(C=k∣X)P(C=k∣X) across classes k and choose the largest. Because P(X)P(X) is the same for all classes, essentially we compute the unnormalized posteriors:
P(C=k)∏i=1nP(xi∣C=k) ,P(C=k)∏i=1n​P(xi​∣C=k) ,
and pick the argmax class
geeksforgeeks.org
geeksforgeeks.org
.

The naive independence assumption says:
P(x1,x2,…,xn∣C=k)=∏i=1nP(xi∣C=k) :contentReference[oaicite:87]index=87.P(x1​,x2​,…,xn​∣C=k)=∏i=1n​P(xi​∣C=k) :contentReference[oaicite:87]index=87.
This drastically simplifies the model, as we just need the distribution of each feature in each class.

To train a Naive Bayes classifier:

    Compute priors: P(C=k)P(C=k) is estimated by frequency (e.g., proportion of training examples in class k)
    geeksforgeeks.org
    .

    Compute likelihoods: For each feature ii and class k, estimate P(xi∣C=k)P(xi​∣C=k). Depending on feature type:

        If xixi​ is categorical (e.g., weather = sunny/overcast/rainy), we use the frequency in class k (with possible Laplace smoothing to handle zero frequencies).

        If xixi​ is continuous, a common approach is to assume a Gaussian Naive Bayes: fit a Gaussian distribution N(μk,i,σk,i2)N(μk,i​,σk,i2​) for feature i in class k, and use its density P(xi∣C=k)=12πσk,iexp⁡(−(xi−μk,i)2/(2σk,i2))P(xi​∣C=k)=2π

        ​σk,i​1​exp(−(xi​−μk,i​)2/(2σk,i2​)). Alternatively, use other distributions if appropriate.

Prediction: Compute scorek=log⁡P(C=k)+∑i=1nlog⁡P(xi∣C=k)scorek​=logP(C=k)+∑i=1n​logP(xi​∣C=k) for each class k, then predict class = argmax_k score_k. (Log is used to avoid underflow and turn products into sums.)

In [ ]:
import numpy as np

# Toy dataset: [speed (mph), spin_rate (rpm)] -> pitch type ('Fastball' or 'Curveball')
data = np.array([
    [95, 2200, 'Fastball'],
    [98, 2100, 'Fastball'],
    [92, 2300, 'Fastball'],
    [75, 2500, 'Curveball'],
    [78, 2700, 'Curveball'],
    [73, 2600, 'Curveball']
], dtype=object)

X = data[:, :2].astype(float)
y = data[:, 2]

# Separate by class
classes = np.unique(y)
class_stats = {}  # will hold prior, mean, var for each class
for c in classes:
    X_c = X[y==c]
    prior = len(X_c) / len(X)
    mean = X_c.mean(axis=0)
    var = X_c.var(axis=0, ddof=1)  # sample variance
    class_stats[c] = {"prior": prior, "mean": mean, "var": var}

# Define Gaussian PDF
def gaussian_pdf(x, mean, var):
    # univariate formula applied element-wise for multi-dim (assumes independence)
    coeff = 1 / np.sqrt(2 * np.pi * var)
    exp = np.exp(- (x-mean)**2 / (2*var))
    return coeff * exp

# Prediction for a new pitch
new_pitch = np.array([90, 2400])  # a pitch with 90 mph speed and 2400 rpm spin
posteriors = {}
for c, stats in class_stats.items():
    # start with log prior
    log_prob = np.log(stats["prior"])
    # add log likelihoods for each feature under Gaussian
    log_prob += np.sum(np.log(gaussian_pdf(new_pitch, stats["mean"], stats["var"])))
    posteriors[c] = log_prob

print("Log-posteriors:", posteriors)
pred_class = max(posteriors, key=posteriors.get)
print(f"Predicted class for pitch {new_pitch}: {pred_class}")


Log-posteriors: {'Curveball': -27.041563918557433, 'Fastball': -11.62369561051438}
Predicted class for pitch [  90 2400]: Fastball


XGBoost (Extreme Gradient Boosting)
Theory – Gradient Boosted Decision Trees

XGBoost is an optimized implementation of gradient boosting decision trees – one of the most successful machine learning techniques in structured data. Gradient Boosting builds an ensemble of trees sequentially, where each new tree corrects errors of the previous ones
geeksforgeeks.org
geeksforgeeks.org
. “Extreme” Gradient Boosting (XGBoost) introduces engineering optimizations and regularization that make it particularly powerful and fast.

Gradient Boosting Recap: Unlike Random Forests (parallel ensemble), boosting is a sequential ensemble:

    Start with an initial model F0(x)F0​(x), often a constant prediction (e.g., mean of targets for regression, or log-odds for classification)
    geeksforgeeks.org
    .

    For each iteration t=1 to Tt=1 to T:

        Compute the residuals (for regression) or negative gradients of the loss function (for general boosting) for each training example. These represent “where the model is getting things wrong”. For example, in regression with MSE loss, the residual is ri=yi−Ft−1(xi)ri​=yi​−Ft−1​(xi​).

        Train a new decision tree ht(x)ht​(x) to predict these residuals (i.e., fit the tree to the errors made so far).

        Add the new tree into the model with a scaling factor: Ft(x)=Ft−1(x)+η ht(x)Ft​(x)=Ft−1​(x)+ηht​(x). Here ηη is the learning rate (shrinkage factor) to control how much each tree contributes
        geeksforgeeks.org
        .

    The final model FT(x)FT​(x) is the sum of all trees’ predictions
    geeksforgeeks.org
    geeksforgeeks.org
    .

Each tree is typically a small CART (often restricted depth), and the boosting process gradually improves performance. Essentially, the model is “learning from mistakes”: each tree addresses the shortcomings of the ensemble so far
geeksforgeeks.org
geeksforgeeks.org
.

XGBoost specifics: XGBoost includes additional features:

    Regularization: It adds regularization terms to the objective to penalize complex trees (e.g., many leaves or large leaf weights)
    geeksforgeeks.org
    geeksforgeeks.org
    . This helps prevent overfitting, making XGBoost often generalize better than an un-regularized boosting.

    Second-order optimization: When fitting each tree, XGBoost uses both first and second derivatives of the loss (if available) to approximate the optimum tree more efficiently (a technique known as Newton boosting).

    Column subsampling: Similar to Random Forest, it can sample features (columns) for each tree or split, adding extra randomness.

    Hardware optimization: XGBoost is optimized in C++ for speed, uses cache-aware block structures, and supports parallelization of certain tasks (though boosting is inherently sequential, tree construction can use multiple cores).

In summary, XGBoost’s objective at each iteration is: find tree ht(x)ht​(x) that maximally improves
objt=∑i=1nl(yi,Ft−1(xi)+ht(xi))+Ω(ht),objt​=∑i=1n​l(yi​,Ft−1​(xi​)+ht​(xi​))+Ω(ht​),
where ll is the loss (e.g., squared error or logistic loss) and Ω(ht)Ω(ht​) is the regularization for the new tree (depends on number of leaves and leaf weights)
geeksforgeeks.org
geeksforgeeks.org
. It uses a greedy algorithm to grow the tree that maximizes this improvement (often using gain formulas derived from gradients and Hessians).
Python Implementation (Toy Example)

Implementing full XGBoost from scratch is very complex. Instead, we illustrate a simple gradient boosting procedure on a small dataset to demonstrate the concept:

Suppose we have a dataset of team ratings (offense and defense) to predict point margin (a regression task: positive means win by that many points, negative means lose). We’ll do a few rounds of boosting with very simple trees (stumps) to see how the ensemble improves predictions step by step.

In [ ]:
# Toy dataset: [offense_rating, defense_rating] -> point_margin (actual score difference)
X = np.array([[8, 6], [5, 7], [6, 6], [3, 4]]) 
y = np.array([10, 2, 5, -3])  # e.g., team1 won by 10, team2 won by 2, etc.

# Initialize model with a prediction (say mean of y)
y_mean = np.mean(y)
pred = np.full_like(y, y_mean, dtype=float)
print("Initial prediction (mean):", pred)

# Iteratively add "trees"
for t in range(2):  # 2 boosting rounds for demo
    residual = y - pred  # compute residuals
    # Fit a simple tree (stump) to residual: we'll just pick one feature and threshold manually for simplicity
    # Determine which feature better splits residuals
    best_feat, best_thresh, best_pred_left, best_pred_right = None, None, None, None
    best_error = float('inf')
    for feat in [0, 1]:
        vals = X[:, feat]
        for thresh in np.unique(vals):
            left_mask = (X[:, feat] <= thresh)
            right_mask = (X[:, feat] > thresh)
            if left_mask.sum() == 0 or right_mask.sum() == 0:
                continue
            # predict residual mean in each region
            pred_left = np.mean(residual[left_mask])
            pred_right = np.mean(residual[right_mask])
            # calculate squared error of this split
            error = np.sum((residual[left_mask] - pred_left)**2) + np.sum((residual[right_mask] - pred_right)**2)
            if error < best_error:
                best_error = error
                best_feat = feat
                best_thresh = thresh
                best_pred_left = pred_left
                best_pred_right = pred_right
    # Update predictions
    update = np.where(X[:, best_feat] <= best_thresh, best_pred_left, best_pred_right)
    pred += update  # add this tree's predictions
    print(f"Round {t+1}: feature{best_feat} <= {best_thresh}, add {{left:{best_pred_left:.1f}, right:{best_pred_right:.1f}}}, new pred = {pred}")


Initial prediction (mean): [3.5 3.5 3.5 3.5]
Round 1: feature0 <= 5, add {left:-4.0, right:4.0}, new pred = [ 7.5 -0.5  7.5 -0.5]
Round 2: feature0 <= 3, add {left:-2.5, right:0.8}, new pred = [ 8.33333333  0.33333333  8.33333333 -3.        ]


Neural Networks (Feedforward)
Theory – Neural Architecture and Backpropagation

Neural networks are inspired by the brain’s neuron connections and are a class of non-linear models that can approximate very complex functions. Here we consider a basic feedforward neural network (also called a multilayer perceptron, MLP) with an input layer, one or more hidden layers, and an output layer.

Architecture: Neurons in each layer take a weighted sum of inputs from the previous layer, apply an activation function, and pass the result to the next layer. For example, a simple 1-hidden-layer network for classification might be:

    Input layer: takes feature vector xx (e.g., a player’s stats).

    Hidden layer: computes h=f(W1x+b1)h=f(W1​x+b1​), where W1W1​ are weights and b1b1​ biases for layer1, and f(⋅)f(⋅) is a non-linear activation (e.g., ReLU or sigmoid). This produces hidden features hh.

    Output layer: computes o=g(W2h+b2)o=g(W2​h+b2​), where gg could be a sigmoid for binary output, softmax for multi-class, or identity for regression. oo gives the network’s output (e.g., predicted probability of win, or predicted points).

Example: If we input a vector of a player’s game stats, the hidden layer might learn abstract features like “fatigue level” or “momentum”, combining raw stats non-linearly, and the output could be a predicted chance of win.

Training – Backpropagation: Neural networks are trained by defining a loss function (e.g., cross-entropy for classification or MSE for regression) and using gradient descent (or advanced optimizers) to adjust weights to minimize the loss. The gradients of the weights are computed by backpropagation, which is essentially the chain rule of calculus applied through the network layers
geeksforgeeks.org
geeksforgeeks.org
:

    Do a forward pass: compute outputs of each layer up to the final output.

    Compute the loss compared to the true label.

    Propagate the error gradient backward:

        Compute gradient at output layer (difference between prediction and target, times derivative of output activation).

        For each hidden layer in reverse, compute its gradient by taking the weighted sum of gradients from the layer above and multiplying by the derivative of its activation function (the chain rule).

    Update weights with gradient descent: W:=W−α∂loss∂WW:=W−α∂W∂loss​ (similarly for biases).

For instance, in a 1-hidden-layer network with sigmoid activations, if a2=σ(W2a1)a2​=σ(W2​a1​) is the output and we have squared error loss, the formula for weight updates would involve terms like (a2−y)σ′(W2a1)(a2​−y)σ′(W2​a1​) for output layer and propagate further to hidden layer weights
geeksforgeeks.org
geeksforgeeks.org
.

The key is that every weight in the network influences the loss and backprop computes those influences efficiently.

Neural networks can capture complex interactions: in sports, this means a NN could learn something like “if player is tall and fast, that combination is especially valuable” through hidden neurons that multiply or interact features. However, NNs need lots of data to train reliably and careful tuning (learning rate, architecture size, etc.) – otherwise they might overfit or get stuck in poor minima.

In [ ]:
# Synthetic dataset: [height, weight] -> position (0=Guard, 1=Forward, 2=Center)
X = np.array([
    [180, 75],  # Guard-like
    [185, 80],  # Guard/Forward borderline
    [195, 90],  # Forward
    [205, 100], # Forward/Center borderline
    [210, 110]  # Center
], dtype=float)
y = np.array([0, 0, 1, 1, 2])  # position labels

# One-hot encode y for 3 classes
Y = np.zeros((len(y), 3))
for i, cls in enumerate(y):
    Y[i, cls] = 1

# Network architecture: 2 -> 3 -> 3 (2 inputs, 3 hidden neurons, 3 output classes)
np.random.seed(0)
W1 = np.random.randn(2, 3) * 0.1  # weight matrix from input to hidden
b1 = np.zeros(3)
W2 = np.random.randn(3, 3) * 0.1  # weight matrix from hidden to output
b2 = np.zeros(3)

# Activation functions
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def softmax(z):
    expz = np.exp(z - np.max(z))
    return expz / expz.sum(axis=1, keepdims=True)

# Training parameters
alpha = 0.1
epochs = 1000

for epoch in range(epochs):
    # Forward pass
    Z1 = X.dot(W1) + b1        # hidden layer linear combination
    A1 = sigmoid(Z1)          # hidden layer activation
    Z2 = A1.dot(W2) + b2      # output layer linear combination
    A2 = softmax(Z2.reshape(len(X), 3))  # output probabilities for 3 classes
    
    # Compute loss (cross-entropy)
    # (for monitoring; not used in backprop explicitly here)
    loss = -np.sum(Y * np.log(A2)) / len(X)
    
    # Backpropagation
    dZ2 = A2 - Y                        # gradient of loss w.rt Z2 (softmax-crossentropy)
    dW2 = A1.T.dot(dZ2) / len(X)
    db2 = np.mean(dZ2, axis=0)
    dA1 = dZ2.dot(W2.T)                # propagate to hidden layer
    dZ1 = dA1 * (A1 * (1 - A1))        # gradient through sigmoid (sigmoid' = A1*(1-A1))
    dW1 = X.T.dot(dZ1) / len(X)
    db1 = np.mean(dZ1, axis=0)
    
    # Gradient descent update
    W2 -= alpha * dW2
    b2 -= alpha * db2
    W1 -= alpha * dW1
    b1 -= alpha * db1

# After training, let's predict the class probabilities
A1 = sigmoid(X.dot(W1) + b1)
A2 = softmax((A1.dot(W2) + b2).reshape(len(X), 3))
print("Predicted probabilities:\n", A2.round(2))
print("Predicted classes:", np.argmax(A2, axis=1))
print("True classes:      ", y)


Predicted probabilities:
 [[0.4 0.4 0.2]
 [0.4 0.4 0.2]
 [0.4 0.4 0.2]
 [0.4 0.4 0.2]
 [0.4 0.4 0.2]]
Predicted classes: [0 0 1 1 1]
True classes:       [0 0 1 1 2]
